In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error;
from sklearn.model_selection import train_test_split;

In [2]:
full_data=pd.read_csv('fulfilment_center_info.csv')
meal_data=pd.read_csv('meal_info.csv')
train_data=pd.read_csv('train.csv')
test_data=pd.read_csv('test.csv')

In [3]:
train_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [4]:
cnt=train_data['num_orders'].values
q99=np.percentile(cnt,[99])


train_df=train_data[train_data['num_orders']<q99[0]]
#sns.distplot(train_df['num_orders'])
#plt.show()

In [5]:
train_data_com=pd.merge(train_df,full_data, how='left',on=['center_id'] )
train_data_com=pd.merge(train_data_com,meal_data, how='left',on=['meal_id'] )

In [6]:
train_data_com['num_orders']=train_data_com['num_orders'].apply(lambda x:np.log(x))

In [7]:
y=train_data_com['num_orders'];

In [8]:
test_data=pd.merge(test_data,full_data, how='left',on=['center_id'] )
test_data=pd.merge(test_data,meal_data, how='left',on=['meal_id'] )

In [9]:
train_data_com.drop('num_orders',axis=1,inplace=True)
print(train_data_com.shape)
print(train_data_com.shape)
print(test_data.shape)

(451953, 14)
(451953, 14)
(32573, 14)


In [10]:
X_ohe=pd.get_dummies(train_data_com);
X_test_ohe=pd.get_dummies(test_data);
#X_test_ohe.head()
print(X_ohe.head());
print(X_test_ohe.head());

        id  week  center_id  meal_id  checkout_price  base_price  \
0  1379560     1         55     1885          136.83      152.29   
1  1466964     1         55     1993          136.83      135.83   
2  1346989     1         55     2539          134.86      135.86   
3  1338232     1         55     2139          339.50      437.53   
4  1448490     1         55     2631          243.50      242.50   

   emailer_for_promotion  homepage_featured  city_code  region_code  \
0                      0                  0        647           56   
1                      0                  0        647           56   
2                      0                  0        647           56   
3                      0                  0        647           56   
4                      0                  0        647           56   

       ...       category_Rice Bowl  category_Salad  category_Sandwich  \
0      ...                        0               0                  0   
1      ...      

In [11]:
train_data_com.head(10)

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,city_code,region_code,center_type,op_area,category,cuisine
0,1379560,1,55,1885,136.83,152.29,0,0,647,56,TYPE_C,2.0,Beverages,Thai
1,1466964,1,55,1993,136.83,135.83,0,0,647,56,TYPE_C,2.0,Beverages,Thai
2,1346989,1,55,2539,134.86,135.86,0,0,647,56,TYPE_C,2.0,Beverages,Thai
3,1338232,1,55,2139,339.50,437.53,0,0,647,56,TYPE_C,2.0,Beverages,Indian
4,1448490,1,55,2631,243.50,242.50,0,0,647,56,TYPE_C,2.0,Beverages,Indian
5,1270037,1,55,1248,251.23,252.23,0,0,647,56,TYPE_C,2.0,Beverages,Indian
6,1191377,1,55,1778,183.36,184.36,0,0,647,56,TYPE_C,2.0,Beverages,Italian
7,1499955,1,55,1062,182.36,183.36,0,0,647,56,TYPE_C,2.0,Beverages,Italian
8,1025244,1,55,2707,193.06,192.06,0,0,647,56,TYPE_C,2.0,Beverages,Italian
9,1054194,1,55,1207,325.92,384.18,0,1,647,56,TYPE_C,2.0,Beverages,Continental


In [12]:
#feature_scaling(test_data['base_price'])
from sklearn.preprocessing import StandardScaler;

scaler=StandardScaler();
X_ohe_scaled = scaler.fit_transform(X_ohe); #X_ohe.copy();
X_test_ohe_scaled = scaler.transform(X_test_ohe); #X_test_ohe.copy();



In [13]:
#X_ohe_scaled=train_data_com.copy();
X_train,X_valid,y_train,y_valid=train_test_split(X_ohe_scaled,y,test_size=0.3,random_state=30);

In [19]:
from sklearn.ensemble import RandomForestRegressor;
rfr=RandomForestRegressor(n_estimators=150,random_state=0);

rfr.fit(X_train,y_train);
rfr_pred=rfr.predict(X_valid);

In [20]:
rfr_pred_res=np.exp(rfr_pred);
rfr_pred_res=rfr_pred_res.astype(np.int64);

y_valid_res=np.exp(y_valid);
y_valid_res=y_valid_res.astype(np.int64);

#print(type(rfr_pred_res));
#print(type(y_valid_res));
print(100*mean_squared_log_error(y_valid_res,rfr_pred_res));
print(np.sum(np.array(rfr_pred_res) >= 0, axis=0));
print(np.sum(np.array(rfr_pred_res) < 0, axis=0));

23.110387456343915
135586
0


In [16]:
#rfr2=RandomForestRegressor(n_estimators=100,random_state=0);
#rfr2.fit(X_ohe_scaled,y);
#test_pred_2=rfr2.predict(X

In [17]:
'''rfr_test_pred=rfr2.predict(X_test_ohe_scaled);
test_pred_res=np.exp(rfr_test_pred);
test_pred=test_pred_res.astype(np.int64);
test_pred'''

'rfr_test_pred=rfr2.predict(X_test_ohe_scaled);\ntest_pred_res=np.exp(rfr_test_pred);\ntest_pred=test_pred_res.astype(np.int64);\ntest_pred'

In [18]:
'''test_pred_ser=pd.Series(test_pred,name="num_orders");
print(type(test_pred));
print(type(test_pred_ser));
print(type(test_data["id"]));
id_test=test_data["id"];
df_rfc = pd.concat([id_test,test_pred_ser],axis=1);
df_rfc.head()
df_rfc.to_csv("submission_rfr_V6.csv",index=False);'''

'test_pred_ser=pd.Series(test_pred,name="num_orders");\nprint(type(test_pred));\nprint(type(test_pred_ser));\nprint(type(test_data["id"]));\nid_test=test_data["id"];\ndf_rfc = pd.concat([id_test,test_pred_ser],axis=1);\ndf_rfc.head()\ndf_rfc.to_csv("submission_rfr_V6.csv",index=False);'

In [24]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [25]:
ridge_m_ = Ridge()
ridge_params_ = { 'max_iter':[3000],'alpha':[0.1, 1, 2, 3, 4, 10, 30,100,200,300,400,800,900,1000]}
rmsle=mean_squared_log_error
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_ridge_m = GridSearchCV( ridge_m_,
                          ridge_params_,
                          scoring = rmsle_scorer,
                          cv=5)
grid_ridge_m.fit(X_train,y_train);
rfr_pred=rfr.predict(X_valid);
grid_preds = grid_ridge_m.predict(X_valid)
print (grid_ridge_m.best_params_)



{'alpha': 0.1, 'max_iter': 3000}


In [26]:
grid_preds_res=np.exp(grid_preds);
grid_preds_res=grid_preds_res.astype(np.int64);

y_valid_res=np.exp(y_valid);
y_valid_res=y_valid_res.astype(np.int64);

In [28]:
print(100*mean_squared_log_error(y_valid_res,grid_preds_res));
print(np.sum(np.array(grid_preds_res) >= 0, axis=0));
print(np.sum(np.array(grid_preds_res) < 0, axis=0));

48.79186466422245
135586
0


In [29]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=100,alpha=0.01); ### Test 0.41
gbm.fit(X_train,y_train)
gbm_pred=gbm.predict(X_valid);

In [30]:
gbm_pred_res=np.exp(gbm_pred);
gbm_pred_res=gbm_pred_res.astype(np.int64);

y_valid_res=np.exp(y_valid);
y_valid_res=y_valid_res.astype(np.int64);

#print(type(rfr_pred_res));
#print(type(y_valid_res));
print(100*mean_squared_log_error(y_valid_res,gbm_pred_res));
print(np.sum(np.array(gbm_pred_res) >= 0, axis=0));
print(np.sum(np.array(gbm_pred_res) < 0, axis=0));

41.69505590087274
135586
0


In [31]:
from sklearn.cross_validation import ShuffleSplit,train_test_split 
from sklearn import metrics 
#from sklearn.learning_curve import learning_curve 
#from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.grid_search import GridSearchCV 


In [39]:
X_train=X_ohe_scaled
y_train=y

In [40]:
def GradientBooster(param_grid, n_jobs):
    estimator = GradientBoostingRegressor()
    cv = ShuffleSplit(X_train.shape[0], n_iter=10, test_size=0.2) 
    classifier = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs) 
    classifier.fit(X_train, y_train)
    print("Best Estimator learned through GridSearch")
    print ("classifier.best_estimator_")
    return cv, classifier.best_estimator_

In [41]:
param_grid={'n_estimators':[100],
            'learning_rate': [0.1,0.05, 0.02, 0.01], 
            'max_depth':[6,4,6], 
            'min_samples_leaf':[3,5,9,17], 
            'max_features':[1.0,0.3,0.1] } 
n_jobs=4

In [ ]:
cv,best_est=GradientBooster(param_grid, n_jobs)